In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('full_results.csv')
df.head()

,data,model,attack,adv_param,defence,acc_no_def,acc_on_adv,fpr
0,banknote,dnn,apgd,0.05,baard,0.766839,0.937824,0.005181
1,banknote,dnn,apgd,0.05,baard,0.767677,0.924242,0.020202
2,banknote,dnn,apgd,0.05,baard,0.764103,0.953846,0.041026
3,banknote,dnn,apgd,0.05,baard,0.777202,0.917098,0.010363
4,banknote,dnn,apgd,0.05,baard,0.783505,0.938144,0.020619


In [3]:
df['defence'].unique()

array(['baard', 'lid', 'magnet', 'rc', 'fs'], dtype=object)

In [4]:
df['attack'].unique()

array(['apgd', 'apgd2', 'cw2', 'fgsm', 'bim', 'boundary', 'tree',
       'deepfool', 'line'], dtype=object)

In [5]:
df_grouped = df.groupby(['data', 'model', 'attack', 'adv_param', 'defence'], as_index=False).agg({
    'acc_no_def': ['mean', 'std'],
    'acc_on_adv': ['mean', 'std'],
    'fpr': ['mean', 'std']
})
df_grouped.columns = ["_".join(x) if x[1] != '' else x[0] for x in df_grouped.columns]
df_grouped = df_grouped.sort_values(by=['data', 'model', 'attack', 'adv_param', 'defence'])
df_grouped

,data,model,attack,adv_param,defence,acc_no_def_mean,acc_no_def_std,acc_on_adv_mean,acc_on_adv_std,fpr_mean,fpr_std
0,banknote,dnn,apgd,0.05,baard,0.771865,0.008171,0.934231,0.014197,0.019478,0.013726
1,banknote,dnn,apgd,0.05,lid,0.771865,0.008171,0.861136,0.024915,0.082222,0.017124
2,banknote,dnn,apgd,0.05,magnet,0.771865,0.008171,0.629780,0.071561,0.003077,0.006880
3,banknote,dnn,apgd,0.05,rc,0.771865,0.008171,0.771897,0.007820,0.001010,0.002259
4,banknote,dnn,apgd,0.10,baard,0.202420,0.030604,0.674106,0.043972,0.019478,0.013726
...,...,...,...,...,...,...,...,...,...,...,...
799,mnist,dnn,line,1.00,baard,0.973600,0.006387,0.994400,0.002793,0.027200,0.003271
800,mnist,dnn,line,1.00,fs,0.973600,0.006387,0.982400,0.005413,0.052800,0.012558
801,mnist,dnn,line,1.00,lid,0.973600,0.006387,0.973600,0.006387,0.000000,0.000000
802,mnist,dnn,line,1.00,magnet,0.973600,0.006387,0.989200,0.006535,0.002800,0.002168


In [6]:
mnist = df_grouped[df_grouped['data'] == 'mnist']
cols = ['acc_no_def_mean', 'acc_no_def_std', 'acc_on_adv_mean', 'acc_on_adv_std', 'fpr_mean', 'fpr_std']
for c in cols:
    mnist[c] = (mnist[c].copy() * 100).round(decimals=1)

mnist


ipykernel_launcher:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,data,model,attack,adv_param,defence,acc_no_def_mean,acc_no_def_std,acc_on_adv_mean,acc_on_adv_std,fpr_mean,fpr_std
679,mnist,dnn,apgd,0.063,baard,86.2,0.4,100.0,0.0,2.7,0.3
680,mnist,dnn,apgd,0.063,fs,86.2,0.4,91.4,1.2,5.3,1.3
681,mnist,dnn,apgd,0.063,lid,86.2,0.4,88.4,1.4,3.6,2.4
682,mnist,dnn,apgd,0.063,magnet,86.2,0.4,100.0,0.0,0.3,0.2
683,mnist,dnn,apgd,0.063,rc,86.2,0.4,89.5,1.0,0.1,0.1
...,...,...,...,...,...,...,...,...,...,...,...
799,mnist,dnn,line,1.000,baard,97.4,0.6,99.4,0.3,2.7,0.3
800,mnist,dnn,line,1.000,fs,97.4,0.6,98.2,0.5,5.3,1.3
801,mnist,dnn,line,1.000,lid,97.4,0.6,97.4,0.6,0.0,0.0
802,mnist,dnn,line,1.000,magnet,97.4,0.6,98.9,0.7,0.3,0.2


In [7]:
res = mnist.pivot(
    index=['attack', 'adv_param'],
    columns=['defence'],
    values=['acc_on_adv_mean', 'acc_on_adv_std']
)
res.to_excel('mnist_acc_on_adv.xlsx')

pd.DataFrame({
    'BAARD': (res[('acc_on_adv_mean',  'baard')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'baard')].astype(str)).values,
    'FS': (res[('acc_on_adv_mean',  'fs')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'fs')].astype(str)).values,
    'LID': (res[('acc_on_adv_mean',  'lid')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'lid')].astype(str)).values,
    'MagNet': (res[('acc_on_adv_mean',  'magnet')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'magnet')].astype(str)).values,
    'RC': (res[('acc_on_adv_mean',  'rc')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'rc')].astype(str)).values,
}).to_excel('mnist_acc_on_adv_2.xlsx')

res = mnist.pivot(
    index=['attack', 'adv_param'],
    columns=['defence'],
    values=['fpr_mean', 'fpr_std']
)
res.to_excel('mnist_fpr.xlsx')

pd.DataFrame({
    'BAARD': (res[('fpr_mean',  'baard')].astype(str) + '$\pm$' + res[('fpr_std',  'baard')].astype(str)).values,
    'FS': (res[('fpr_mean',  'fs')].astype(str) + '$\pm$' + res[('fpr_std',  'fs')].astype(str)).values,
    'LID': (res[('fpr_mean',  'lid')].astype(str) + '$\pm$' + res[('fpr_std',  'lid')].astype(str)).values,
    'MagNet': (res[('fpr_mean',  'magnet')].astype(str) + '$\pm$' + res[('fpr_std',  'magnet')].astype(str)).values,
    'RC': (res[('fpr_mean',  'rc')].astype(str) + '$\pm$' + res[('fpr_std',  'rc')].astype(str)).values,
}).to_excel('mnist_fpr_2.xlsx')

In [8]:
cifar10 = df_grouped[(df_grouped['data'] == 'cifar10') & (df_grouped['model'] == 'resnet')]
cols = ['acc_no_def_mean', 'acc_no_def_std', 'acc_on_adv_mean', 'acc_on_adv_std', 'fpr_mean', 'fpr_std']
for c in cols:
    cifar10[c] = (cifar10[c].copy() * 100).round(decimals=1)

res = cifar10.pivot(
    index=['attack', 'adv_param'],
    columns=['defence'],
    values=['acc_on_adv_mean', 'acc_on_adv_std']
)
res.to_excel('cifar10_acc_on_adv.xlsx')

pd.DataFrame({
    'BAARD': (res[('acc_on_adv_mean',  'baard')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'baard')].astype(str)).values,
    'FS': (res[('acc_on_adv_mean',  'fs')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'fs')].astype(str)).values,
    'LID': (res[('acc_on_adv_mean',  'lid')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'lid')].astype(str)).values,
    'MagNet': (res[('acc_on_adv_mean',  'magnet')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'magnet')].astype(str)).values,
    'RC': (res[('acc_on_adv_mean',  'rc')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'rc')].astype(str)).values,
}).to_excel('cifar10_acc_on_adv_2.xlsx')

res = cifar10.pivot(
    index=['attack', 'adv_param'],
    columns=['defence'],
    values=['fpr_mean', 'fpr_std']
)
res.to_excel('cifar10_fpr.xlsx')

pd.DataFrame({
    'BAARD': (res[('fpr_mean',  'baard')].astype(str) + '$\pm$' + res[('fpr_std',  'baard')].astype(str)).values,
    'FS': (res[('fpr_mean',  'fs')].astype(str) + '$\pm$' + res[('fpr_std',  'fs')].astype(str)).values,
    'LID': (res[('fpr_mean',  'lid')].astype(str) + '$\pm$' + res[('fpr_std',  'lid')].astype(str)).values,
    'MagNet': (res[('fpr_mean',  'magnet')].astype(str) + '$\pm$' + res[('fpr_std',  'magnet')].astype(str)).values,
    'RC': (res[('fpr_mean',  'rc')].astype(str) + '$\pm$' + res[('fpr_std',  'rc')].astype(str)).values,
}).to_excel('cifar10_fpr_2.xlsx')

ipykernel_launcher:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
cifar10 = df_grouped[(df_grouped['data'] == 'cifar10') & (df_grouped['model'] == 'vgg')]
cols = ['acc_no_def_mean', 'acc_no_def_std', 'acc_on_adv_mean', 'acc_on_adv_std', 'fpr_mean', 'fpr_std']
for c in cols:
    cifar10[c] = (cifar10[c].copy() * 100).round(decimals=1)

res = cifar10.pivot(
    index=['attack', 'adv_param'],
    columns=['defence'],
    values=['acc_on_adv_mean', 'acc_on_adv_std']
)
res.to_excel('cifar10_acc_on_adv.xlsx')

pd.DataFrame({
    'BAARD': (res[('acc_on_adv_mean',  'baard')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'baard')].astype(str)).values,
    'FS': (res[('acc_on_adv_mean',  'fs')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'fs')].astype(str)).values,
    'LID': (res[('acc_on_adv_mean',  'lid')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'lid')].astype(str)).values,
    'MagNet': (res[('acc_on_adv_mean',  'magnet')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'magnet')].astype(str)).values,
    'RC': (res[('acc_on_adv_mean',  'rc')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'rc')].astype(str)).values,
}).to_excel('cifar10_vgg_acc_on_adv_2.xlsx')

res = cifar10.pivot(
    index=['attack', 'adv_param'],
    columns=['defence'],
    values=['fpr_mean', 'fpr_std']
)
res.to_excel('cifar10_fpr.xlsx')

pd.DataFrame({
    'BAARD': (res[('fpr_mean',  'baard')].astype(str) + '$\pm$' + res[('fpr_std',  'baard')].astype(str)).values,
    'FS': (res[('fpr_mean',  'fs')].astype(str) + '$\pm$' + res[('fpr_std',  'fs')].astype(str)).values,
    'LID': (res[('fpr_mean',  'lid')].astype(str) + '$\pm$' + res[('fpr_std',  'lid')].astype(str)).values,
    'MagNet': (res[('fpr_mean',  'magnet')].astype(str) + '$\pm$' + res[('fpr_std',  'magnet')].astype(str)).values,
    'RC': (res[('fpr_mean',  'rc')].astype(str) + '$\pm$' + res[('fpr_std',  'rc')].astype(str)).values,
}).to_excel('cifar10_vgg_fpr_2.xlsx')

ipykernel_launcher:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
data = df_grouped[((df_grouped['data'] == 'banknote') | (df_grouped['data'] == 'htru2') | (df_grouped['data'] == 'breastcancer')) & (df_grouped['model'] == 'dnn')]
cols = ['acc_no_def_mean', 'acc_no_def_std', 'acc_on_adv_mean', 'acc_on_adv_std', 'fpr_mean', 'fpr_std']
for c in cols:
    data[c] = (data[c].copy() * 100).round(decimals=1)

print(data['defence'].unique())

res = data.pivot(
    index=['data', 'attack', 'adv_param'],
    columns=['defence'],
    values=['acc_on_adv_mean', 'acc_on_adv_std']
)
res.to_excel('num_acc_on_adv.xlsx')

pd.DataFrame({
    'BAARD': (res[('acc_on_adv_mean',  'baard')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'baard')].astype(str)).values,
    'LID': (res[('acc_on_adv_mean',  'lid')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'lid')].astype(str)).values,
    'MagNet': (res[('acc_on_adv_mean',  'magnet')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'magnet')].astype(str)).values,
    'RC': (res[('acc_on_adv_mean',  'rc')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'rc')].astype(str)).values,
}).to_excel('num_acc_on_adv_2.xlsx')

res = data.pivot(
    index=['data', 'attack', 'adv_param'],
    columns=['defence'],
    values=['fpr_mean', 'fpr_std']
)
res.to_excel('num_fpr.xlsx')

pd.DataFrame({
    'BAARD': (res[('fpr_mean',  'baard')].astype(str) + '$\pm$' + res[('fpr_std',  'baard')].astype(str)).values,
    'LID': (res[('fpr_mean',  'lid')].astype(str) + '$\pm$' + res[('fpr_std',  'lid')].astype(str)).values,
    'MagNet': (res[('fpr_mean',  'magnet')].astype(str) + '$\pm$' + res[('fpr_std',  'magnet')].astype(str)).values,
    'RC': (res[('fpr_mean',  'rc')].astype(str) + '$\pm$' + res[('fpr_std',  'rc')].astype(str)).values,
}).to_excel('num_fpr_2.xlsx')

ipykernel_launcher:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
['baard' 'lid' 'magnet' 'rc']


In [11]:
data = df_grouped[((df_grouped['data'] == 'banknote') | (df_grouped['data'] == 'htru2') | (df_grouped['data'] == 'breastcancer')) & (df_grouped['model'] == 'svm')]
cols = ['acc_no_def_mean', 'acc_no_def_std', 'acc_on_adv_mean', 'acc_on_adv_std', 'fpr_mean', 'fpr_std']
for c in cols:
    data[c] = (data[c].copy() * 100).round(decimals=1)

print(data['defence'].unique())

res = data.pivot(
    index=['data', 'attack', 'adv_param'],
    columns=['defence'],
    values=['acc_on_adv_mean', 'acc_on_adv_std']
)
res.to_excel('svm_acc_on_adv.xlsx')

pd.DataFrame({
    'BAARD': (res[('acc_on_adv_mean',  'baard')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'baard')].astype(str)).values,
    'MagNet': (res[('acc_on_adv_mean',  'magnet')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'magnet')].astype(str)).values,
    'RC': (res[('acc_on_adv_mean',  'rc')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'rc')].astype(str)).values,
}).to_excel('svm_acc_on_adv_2.xlsx')

res = data.pivot(
    index=['data', 'attack', 'adv_param'],
    columns=['defence'],
    values=['fpr_mean', 'fpr_std']
)
res.to_excel('svm_fpr.xlsx')

pd.DataFrame({
    'BAARD': (res[('fpr_mean',  'baard')].astype(str) + '$\pm$' + res[('fpr_std',  'baard')].astype(str)).values,
    'MagNet': (res[('fpr_mean',  'magnet')].astype(str) + '$\pm$' + res[('fpr_std',  'magnet')].astype(str)).values,
    'RC': (res[('fpr_mean',  'rc')].astype(str) + '$\pm$' + res[('fpr_std',  'rc')].astype(str)).values,
}).to_excel('svm_fpr_2.xlsx')

ipykernel_launcher:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
['baard' 'magnet' 'rc']


In [12]:
data = df_grouped[((df_grouped['data'] == 'banknote') | (df_grouped['data'] == 'htru2') | (df_grouped['data'] == 'breastcancer')) & (df_grouped['model'] == 'tree')]
cols = ['acc_no_def_mean', 'acc_no_def_std', 'acc_on_adv_mean', 'acc_on_adv_std', 'fpr_mean', 'fpr_std']
for c in cols:
    data[c] = (data[c].copy() * 100).round(decimals=1)

print(data['defence'].unique())

res = data.pivot(
    index=['data', 'attack', 'adv_param'],
    columns=['defence'],
    values=['acc_on_adv_mean', 'acc_on_adv_std']
)
res.to_excel('tree_acc_on_adv.xlsx')

pd.DataFrame({
    'BAARD': (res[('acc_on_adv_mean',  'baard')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'baard')].astype(str)).values,
    'MagNet': (res[('acc_on_adv_mean',  'magnet')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'magnet')].astype(str)).values,
    'RC': (res[('acc_on_adv_mean',  'rc')].astype(str) + '$\pm$' + res[('acc_on_adv_std',  'rc')].astype(str)).values,
}).to_excel('tree_acc_on_adv_2.xlsx')

res = data.pivot(
    index=['data', 'attack', 'adv_param'],
    columns=['defence'],
    values=['fpr_mean', 'fpr_std']
)
res.to_excel('tree_fpr.xlsx')

pd.DataFrame({
    'BAARD': (res[('fpr_mean',  'baard')].astype(str) + '$\pm$' + res[('fpr_std',  'baard')].astype(str)).values,
    'MagNet': (res[('fpr_mean',  'magnet')].astype(str) + '$\pm$' + res[('fpr_std',  'magnet')].astype(str)).values,
    'RC': (res[('fpr_mean',  'rc')].astype(str) + '$\pm$' + res[('fpr_std',  'rc')].astype(str)).values,
}).to_excel('tree_fpr_2.xlsx')

ipykernel_launcher:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
['baard' 'magnet' 'rc']
